# 7장 데이터 준비하기: 다듬기, 변형, 병합
## 문자열 다루기

In [2]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

## 1. 문자열 객체 메서드

In [2]:
val = 'a,b, guido'
val

'a,b, guido'

In [3]:
val.split(',')

['a', 'b', ' guido']

- 기본적으로는 split 메서드 사용

In [4]:
piece = [x.strip() for x in val.split(',')]
piece

['a', 'b', 'guido']

- strip 메서드로 불필요한 문자 제거

In [5]:
first, second, third = piece
first + '::' + second + '::' + third

'a::b::guido'

In [6]:
'::'.join(piece)

'a::b::guido'

- 문자열은 +연산을 통해 합칠수도 있음
- 문자열은 문자.join()메서드로 합칠 수도 있음

In [10]:
val

'a,b, guido'

In [7]:
'guido' in val

True

In [8]:
val.index(',')

1

In [9]:
val.find(':')

-1

In [11]:
val.index(':')

ValueError: substring not found

In [12]:
val.find(',')

1

- index와 find 메서드로 문자열 검색 가능
- 또는 in 예약어로도 검색 가능

In [13]:
val.count(',')

2

In [15]:
val.replace(',', '::')

'a::b:: guido'

In [16]:
val.replace(',','')

'ab guido'

- replace는 찾아낸 패턴을 다른 문자로 치환
- 하단은 다양한 내장 문자열 메서드(유용)

| 인자 | 설명 |
| ---- | ---- |
| count | 겹치지 않는 부분 문자열의 개수 반환|
| endswith, startwith | 문자열이 주어진 접미사(접두사)로 끝날 경우 True 반환|
| join|문자열을 구분자로, 다른 문자열을 순서대로 결합|
|index|문자열의 첫번째 글자 위치 반환, 없다면 ValueError 발생|
|find|문자열의 첫번째 글자 위치 반환, 없다면 -1 반환|
|rfind| 마지막 문자열의 첫번째 글자의 위치 반환, 문자열이 없다면 -1 반환|
|replace| 문자열을 다른 문자열로 치환|
|strip, lstrip, rstrip| 문자열 제거|
|split| 구분자 기준으로 분리 (리스트)|
|lower, upper| 알파벳을 소문자/대문자로 변환|
|ljust, rjust| 문자열을 왼쪽/오른쪽 정렬 후 주어진 길이 제외한 구간은 공백문자 삽입|

___
## 2. 정규표현식

In [1]:
import re

In [3]:
text = 'foo     bar\t baz   \tqux'
text

'foo     bar\t baz   \tqux'

In [19]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

- 정규표현식 '\s+'는 하나 이상의 공백문자를 의미

In [4]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

- 정규표현식을 compile 하고 정규표현식 객체를 재사용 하는 것도 가능

In [6]:
regex.findall(text)

['     ', '\t ', '   \t']

- 정규표현식 객체.findall()로 매칭되는 모든 패턴 목록 반환 가능

In [35]:
text = '''
Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@gmail.com
'''

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

regex = re.compile(pattern, flags = re.IGNORECASE)

In [36]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@gmail.com']

In [38]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(6, 21), match='dave@google.com'>

In [39]:
text[m.start():m.end()]

'dave@google.com'

- search는 텍스트에서 첫번째 이메일 주소만 반환
- match 객체는 문자열 안에서 위치하는 시작점과 끝점만 반환

In [18]:
print(regex.match(text))

None


In [40]:
print(regex.sub('REDACTED', text))


Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



- sub 메서드는 찾은 패턴을 주어진 문자열로 치환

In [14]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags = re.IGNORECASE)

In [46]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [47]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'gmail', 'com')]

- match 객체를 이용, groups 메서드를 통해 튜플 반환 가능

In [48]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))


Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: gmail, Suffix: com



- \1, \2 같은 특수 기호로 각 패턴의 그룹에 접근 가능
___

In [5]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags = re.IGNORECASE|re.VERBOSE)

In [6]:
m = regex.match('wesm@bright.net')
m.groupdict()

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

- 위의 예처럼 match 그룹에 이름을 부여하는 것도 가능
- (?P<이름>......
___
## 3. pandas의 벡터화된 문자열 함수

In [7]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@google.com',
       'Rob': 'rob@google.com', 'Wes': np.nan}
data = Series(data)
data

Dave      dave@google.com
Rob        rob@google.com
Steve    steve@google.com
Wes                   NaN
dtype: object

In [11]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

- NA 값을 만날 때 실패할 수 있음
- 하지만 Series에는 NA 값을 건너 뛰게 하는 메서드가 있음

In [12]:
data.str.contains('gmail')

Dave     False
Rob      False
Steve    False
Wes        NaN
dtype: object

In [15]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [16]:
data.str.findall(pattern, flags = re.IGNORECASE)

Dave      [(dave, google, com)]
Rob        [(rob, google, com)]
Steve    [(steve, google, com)]
Wes                         NaN
dtype: object

- ignorecase 같은 정규표현식도 re와 함께 사용 가능

In [17]:
matches = data.str.match(pattern, flags = re.IGNORECASE)
matches

Dave     True
Rob      True
Steve    True
Wes       NaN
dtype: object

In [18]:
matches.str.get(1)

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64